In [1]:
import os
import codecs

In [2]:
# get google cloud
!pip install --upgrade google-cloud-translate
from google.cloud import translate

Requirement already up-to-date: google-cloud-translate in c:\programdata\anaconda3\lib\site-packages (1.3.3)


In [26]:
# get nltk to calculate bleu score
!pip install --upgrade nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

Requirement already up-to-date: nltk in c:\programdata\anaconda3\lib\site-packages (3.4)


# Zulu

In [65]:
import codecs

# laod data
en_file = codecs.open('../clean/en_zu/enzu_parallel.test.en', 'r')
zu_file = codecs.open('../clean/en_zu/enzu_parallel.test.zu', 'r')

en_sents = [line.rstrip('\n') for line in en_file.readlines()]
zu_sents = [line.rstrip('\n') for line in zu_file.readlines()]

# translate
if not os.path.exists('./translated.zu'):
    trans_file = codecs.open('./translated.zu','w')
    
    # translate 100 sentences at a time
    translate_client = translate.Client()
    target = 'zu'
    for i in range(0, len(en_sents) // 100):
        translations = translate_client.translate(
            en_sents[i:i+100],
            target_language=target,
            source_language='en'
        )

        for translation in translations:
            trans_file.write(translation['translatedText'])
            trans_file.write('\n')
        
    trans_file.close()
    
# calculate bleu score
trans_file = codecs.open('./translated.zu', 'r')
trans_sents = [line.rstrip('\n').split(' ') for line in trans_file.readlines()]
num_trans = len(trans_sents)

zu_refs = [[line.split(' ')] for line in zu_sents]

bleu_score = corpus_bleu(zu_refs[:num_trans], trans_sents)

print(bleu_score*100)

trans_file.close()
en_file.close()
zu_fie.clos

0.3292056420937072
